# Setting up SDK artifacts

In [ ]:
## Give appropriate permission to the directory "FOLDER_WITH_ARTIFACTS" you are working with
import os
os.environ['SNPE_ROOT']="/local/mnt/workspace/aditya/qaisw-v2.15.1.230926150623_62883"#set up your snpe path here.
os.environ['RAW_FILE_FOLDER']="input/raw"
os.environ['FOLDER_WITH_ARTIFACTS']="../FFNet/"
os.environ['DLC32']="models/ffnet_fp32.dlc"
os.environ['DLC8']="models/ffnet_w8a8.dlc"
os.environ['TARGET_INPUT_LIST']="input/input.txt"
os.environ['ONDEVICE_FOLDER']="ffnetseg"
os.environ['DEVICE_HOST']="localhost"
os.environ['DEVICE_ID']="2dce6316" #change with your device-id. Use command "adb devices" to get devices names.
os.environ['SNPE_TARGET_ARCH']="aarch64-android"
os.environ['SNPE_TARGET_STL']="libc++_shared.so"

In [ ]:
%%bash
git clone https://github.com/quic/aimet-model-zoo/

In [ ]:
%%bash
cp -r ffnet.patch aimet-model-zoo/
cd aimet-model-zoo/
git apply ffnet.patch

In [ ]:
import os
os.makedirs('utils', exist_ok=True)

In [ ]:
%%bash
cp -r aimet-model-zoo/aimet_zoo_torch/ffnet/model/* ./utils/
rm -rf aimet-model-zoo/

In [ ]:
%%bash
wget https://github.com/quic/aimet-model-zoo/releases/download/torch_segmentation_ffnet/ffnet40S_dBBB_cityscapes_state_dict_quarts.pth

In [ ]:
from utils.model_registry import model_entrypoint
net = model_entrypoint("segmentation_ffnet40S_dBBB_mobile")()
net.eval()

# Getting The ONNX Model

In [ ]:
os.makedirs('models',exist_ok=True)

In [ ]:
import torch
import os
import cv2
import glob
import numpy as np
import torch
from os.path import isfile, join
import matplotlib.pyplot as plt 
from PIL import Image
from torchvision import transforms as T

In [ ]:
dummy_input = torch.randn(1,3, 512, 512).type(torch.FloatTensor).to('cpu')
torch.onnx.export(net, dummy_input, "./models/ffnet.onnx",opset_version=11)

### Generate DLC from ONNX

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-onnx-to-dlc --input_network models/ffnet.onnx --output_path models/ffnet_fp32.dlc

## Pre-Process the data

In [ ]:
os.makedirs('input',exist_ok=True)
os.makedirs('input/dataset',exist_ok=True)
os.makedirs('input/dataset/image/',exist_ok=True)
os.makedirs('input/dataset/ground_truth/',exist_ok=True)

## Steps to set Dataset path


User needs to download the dataset of their choice

## Generate raw files

In [ ]:
os.makedirs('input/raw', exist_ok=True)

In [ ]:
test_images_dir = 'input/dataset/image/'
all_files = os.listdir(test_images_dir)
img_paths = []
image_names = []
for file in all_files:
    img_paths.append(test_images_dir+file)
    name = file.replace(".png", "")
    image_names.append(name)
img_paths = sorted(img_paths)
image_names.sort()
i=0
for img in img_paths:
    raw_data = preProcess(img)
    type(raw_data)
    raw_data.tofile('input/raw/'+image_names[i]+'.raw')
    i = i+1

In [ ]:
directory_path = 'input/raw/'
output_file_path = 'input/input.txt'  # The file where the output will be saved
all_files = os.listdir(directory_path)
# Filter only the .raw files and create a list of their names
raw_files = [file for file in all_files if file.endswith('.raw')]
raw_files = sorted(raw_files)
# Write the file names to the output file
with open(output_file_path, 'w') as f:
    c=0
    for raw_file in raw_files:
        f.write(f"./raw/{raw_file}\n")
        c=c+1
print(f"File names written to {output_file_path}.")

## generate Quantized dlc from dlc

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
cd input
snpe-dlc-quantize --input_dlc ../models/ffnet_fp32.dlc --input_list input.txt --output_dlc ../models/ffnet_w8a8.dlc 

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
cd input
snpe-dlc-quantize --input_dlc ../models/ffnet_fp32.dlc --input_list input.txt --output_dlc ../models/ffnet_w8a16.dlc --act_bitwidth 16 

In [ ]:
%%bash
adb devices

## Model Inference on Device

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/dsp/lib"

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/$SNPE_TARGET_STL /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib
$DEVICE_SHELL push $SNPE_ROOT/bin/$SNPE_TARGET_ARCH/snpe-net-run /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin
$DEVICE_SHELL push $SNPE_ROOT/lib/hexagon-v75/unsigned/*.so /data/local/tmp/snpeexample/dsp/lib
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/*.so /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/$ONDEVICE_FOLDER"

## Push files on device

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $DLC32 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $DLC8 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $RAW_FILE_FOLDER /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $TARGET_INPUT_LIST /data/local/tmp/$ONDEVICE_FOLDER

## Inferencing the FP32 Model

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export OUTPUT_FOLDER=OUTPUT_32b_CPU
export OUTPUT_DLC_32=ffnet_fp32.dlc
export ONDEVICE_FOLDER="ffnetseg"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_32 --input_list input.txt --output_dir $OUTPUT_FOLDER 

## Inferencing the INT 8 Model

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export OUTPUT_FOLDER=OUTPUT_8b_DSP
export OUTPUT_DLC_QUANTIZED8=ffnet_w8a8.dlc
export ADSP_LIBRARY_PATH="/data/local/tmp/snpeexample/dsp/lib;/system/lib/rfsa/adsp;/system/vendor/lib/rfsa/adsp;/dsp"
export ONDEVICE_FOLDER="ffnetseg"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_QUANTIZED8 --input_list input.txt --output_dir $OUTPUT_FOLDER --use_dsp

## Pull output raw files 

In [ ]:
os.makedirs('output',exist_ok=True)

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_8b_DSP output/OUTPUT_8b_DSP
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_32b_CPU output/OUTPUT_32b_CPU

# Post Processing

#### Creating the Necessary Path Needed

In [ ]:
if not os.path.exists('output/model_prediction'):
    os.makedirs('output/model_prediction')
    
if not os.path.exists('output/test_results/'):
    os.makedirs('output/test_results/')
    
if not os.path.exists('output/test_results/8b_dsp'):
    os.makedirs('output/test_results/8b_dsp')
    
if not os.path.exists('output/test_results/32b_arm'):
    os.makedirs('output/test_results/32b_arm')


## function for post processing raw outputs

In [ ]:
def postProcessing(img_path,out_path):
    res = np.fromfile(img_path, dtype="float32")
    #print("res", res.shape)
    res_reshape = res.reshape((1,64,64,19)).astype(np.float32)
    res_reshape = np.transpose(res_reshape,(0,3,1,2))
    res_reshape = torch.from_numpy(res_reshape)
    size = [1,3,512,1024]
    pred = F.interpolate(
            input=res_reshape, size=size[-2:],
            mode='bilinear', align_corners=False
        )
    model_img = pred
    pred = model_img.max(1)[1].cpu().numpy()[0] # HW
    pred.shape

    plt.imsave(out_path,pred)

In [ ]:
from torch.nn import functional as F
import matplotlib.pyplot as plt
import os
test_images_dir = "output/OUTPUT_8b_DSP/"

for i in range(0,5):
    img_path = os.path.join(test_images_dir, 'Result_')
    img_path = img_path+str(i)+'/617.raw'
    out_path = 'output/test_results/8b_dsp/'+image_names[i]+'.png'
    postProcessing(img_path, out_path)
    i = i +1

In [ ]:
from torch.nn import functional as F
import matplotlib.pyplot as plt

test_images_dir = "output/OUTPUT_32b_CPU/"

for i in range(0,5):
    img_path = os.path.join(test_images_dir, 'Result_')
    img_path = img_path+str(i)+'/617.raw'
    out_path = 'output/test_results/32b_arm/'+image_names[i]+'.png'
    postProcessing(img_path, out_path)
    i = i +1

In [ ]:
def image_overlay(image, segmented_image):
 
    alpha = 0.6 # how much transparency to apply
    beta = 1 - alpha # alpha + beta should equal 1
    gamma = 0 # scalar added to each sum
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    segmented_image = cv2.cvtColor(segmented_image, cv2.COLOR_RGB2BGR)
    cv2.addWeighted(segmented_image, alpha, image, beta, gamma, image)
    return image

## Visualize the outputs

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(30, 100));
import cv2
for i in range(0, 5):
    
    original = cv2.imread('input/dataset/image/'+image_names[i]+'.png')
    original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    ax = fig.add_subplot(28,4,4*i+1);
    plt.imshow(original,cmap='gray');
    ax.set_title('original image\n');
    ax.axis('off');

    temp_name = image_names[i].replace("_leftImg8bit","_gtFine_color")
    pth_inf = cv2.imread('input/dataset/ground_truth/'+temp_name+'.png')

    ax = fig.add_subplot(28,4,4*i+2);
    plt.imshow(pth_inf,cmap='gray');
    ax.set_title('pth output\n');
    ax.axis('off');


    arm_fp32= cv2.imread('output/test_results/32b_arm/'+image_names[i]+'.png')

    ax = fig.add_subplot(28,4,4*i+3);
    plt.imshow(arm_fp32,cmap='gray');
    ax.set_title('fp32 on ARM\n');
    ax.axis('off');

    dsp= cv2.imread('output/test_results/8b_dsp/'+image_names[i]+'.png')
    ax = fig.add_subplot(28,4,4*i+4);
    plt.imshow(dsp,cmap='gray');
    ax.set_title('int8 on DSP\n');
    ax.axis('off');